## sklearn-crfsuite

In this notebook we train a basic CRF model for Named Entity Recognition on CoNLL2002 data (following https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb) and check its weights to see what it learned.

In [1]:
import sys
sys.path.insert(0, '..')

import nltk
import sklearn_crfsuite

from eli5 import explain_weights, format_as_text
from eli5.formatters import fields

Load training data:

In [2]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

CPU times: user 2.86 s, sys: 136 ms, total: 2.99 s
Wall time: 3.06 s


Extract features: word parts, POS tags, lower/title/upper flags, features of nearby words

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [4]:
sent2features(train_sents[0])[0]

{'+1:postag': 'Fpa',
 '+1:postag[:2]': 'Fp',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': '(',
 'BOS': True,
 'bias': 1.0,
 'postag': 'NP',
 'postag[:2]': 'NP',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'melbourne',
 'word[-2:]': 'ne',
 'word[-3:]': 'rne'}

In [5]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 1.55 s, sys: 150 ms, total: 1.7 s
Wall time: 1.73 s


Train a CRF model:

In [6]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20, 
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

CPU times: user 15.1 s, sys: 269 ms, total: 15.4 s
Wall time: 16 s


Check CRF weights (transition weights and state weights):

In [7]:
explain_weights(crf, top=20)

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='B-LOC', feature_weights=FeatureWeights(pos=[('word.istitle()', 2.3082630000000002), ('-1:word.lower():en', 2.21529), ('word[-2:]:id', 0.96784000000000003), ('word[-3:]:rid', 0.92051000000000005), ('word.lower():madrid', 0.91972699999999996), ('word[-2:]:ia', 0.78586400000000001), ('word[-2:]:ña', 0.68621699999999997), ('word[-2:]:ís', 0.67472399999999999), ('word[-3:]:ona', 0.66492799999999996), ('word.lower():españa', 0.64580499999999996), ('-1:postag[:2]:Fp', 0.59740499999999996), ('-1:word.lower():(', 0.58201800000000004), ('-1:postag:Fpa', 0.58201800000000004), ('postag:NP', 0.56610000000000005), ('postag[:2]:NP', 0.56610000000000005)], neg=[('-1:word.istitle()', -1.3437209999999999), ('postag[:2]:SP', -1.011741), ('postag:SP', -1.011741), ('-1:word.lower():del', -0.83347599999999999), ('word[-3:]:ión', -0.625556)], pos_remaining=2428, neg_remaining=506), proba=None, score=None, weighted_spans=None), TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[('postag[:2]:Fp', 3.643554), ('BOS', 3.5770309999999998), ('bias', 3.1395499999999998), ('postag:CC', 1.998208), ('postag[:2]:CC', 1.998208), ('word[-2:]:,', 1.8384199999999999), ('word.lower():,', 1.8384199999999999), ('word[-3:]:,', 1.8384199999999999), ('postag:Fc', 1.8384199999999999), ('postag[:2]:Fc', 1.8384199999999999), ('word.lower():y', 1.7064630000000001), ('EOS', 1.559275), ('postag:RG', 1.4515929999999999), ('postag[:2]:RG', 1.4515929999999999), ('postag[:2]:Fg', 1.278837), ('word.lower():-', 1.278837)], neg=[('word.istitle()', -6.5745909999999999), ('word.isupper()', -4.1145529999999999), ('postag[:2]:NP', -2.1962449999999998), ('postag:NP', -2.1962449999999998)], pos_remaining=16970, neg_remaining=4166), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-ORG', feature_weights=FeatureWeights(pos=[('word.lower():efe', 2.4133260000000001), ('word.isupper()', 2.1558670000000002), ('word[-2:]:FE', 1.1325320000000001), ('word[-3:]:EFE', 1.1172089999999999), ('word.lower():gobierno', 1.1032580000000001), ('-1:word.lower():del', 0.96011999999999997), ('word.istitle()', 0.88195299999999999), ('word[-3:]:rno', 0.84985699999999997), ('-1:word.lower():al', 0.77758499999999997), ('word[-2:]:PP', 0.74130200000000002), ('word.lower():pp', 0.73424999999999996), ('word[-3:]:PP', 0.73424999999999996), ('-1:postag[:2]:DA', 0.61978500000000003), ('-1:postag:DA', 0.61978500000000003)], neg=[('postag:SP', -1.417144), ('postag[:2]:SP', -1.417144), ('-1:word.lower():en', -0.73296300000000003), ('word[-2:]:es', -0.71601700000000001), ('-1:word.lower():de', -0.71592699999999998), ('-1:postag[:2]:VM', -0.68142100000000005)], pos_remaining=3848, neg_remaining=721), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-PER', feature_weights=FeatureWeights(pos=[('word.istitle()', 2.8049430000000002), ('postag:NP', 0.78541799999999995), ('postag[:2]:NP', 0.78541799999999995), ('+1:postag:VMI', 0.60152099999999997), ('-1:word.lower():a', 0.59059099999999998), ('+1:postag[:2]:VM', 0.59041100000000002)], neg=[('postag[:2]:DA', -1.165368), ('postag:DA', -1.165368), ('-1:word.istitle()', -1.024311), ('-1:postag[:2]:DA', -1.0027159999999999), ('-1:postag:DA', -1.0027159999999999), ('-1:word.lower():del', -0.85910299999999995), ('-1:word.lower():la', -0.74174700000000005), ('word[-2:]:os'

Let's check how CRF's all_possible_transitions argument affects the result:

In [8]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20, 
    all_possible_transitions=True,
)
crf.fit(X_train, y_train)

CPU times: user 15.6 s, sys: 134 ms, total: 15.7 s
Wall time: 15.8 s


In [9]:
expl = explain_weights(crf, top=20)
expl

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='B-LOC', feature_weights=FeatureWeights(pos=[('-1:word.lower():en', 2.174998), ('word.istitle()', 1.9254709999999999), ('word[-2:]:id', 0.99532100000000001), ('word[-3:]:rid', 0.94915799999999995), ('word.lower():madrid', 0.94913899999999995), ('word[-2:]:ís', 0.69411599999999996), ('+1:postag[:2]:Fp', 0.68532300000000002), ('word[-3:]:ona', 0.67401999999999995), ('word[-2:]:ña', 0.65928100000000001), ('word.lower():españa', 0.65370499999999998), ('BOS', 0.55398000000000003), ('word[-2:]:ia', 0.54925100000000004), ('word.lower():parís', 0.54295199999999999), ('word[-3:]:rís', 0.54131600000000002), ('word[-3:]:aña', 0.53118200000000004)], neg=[('-1:word.istitle()', -0.96316599999999997), ('postag[:2]:SP', -0.92849199999999998), ('postag:SP', -0.92849199999999998), ('word[-3:]:ión', -0.67950900000000003), ('-1:word.lower():del', -0.63449100000000003)], pos_remaining=2430, neg_remaining=505), proba=None, score=None, weighted_spans=None), TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[('postag[:2]:Fp', 3.8396919999999999), ('BOS', 3.4126970000000001), ('bias', 2.9151069999999999), ('postag:CC', 2.1810170000000002), ('postag[:2]:CC', 2.1810170000000002), ('EOS', 1.9229639999999999), ('word.lower():y', 1.8781589999999999), ('word[-3:]:,', 1.815958), ('word.lower():,', 1.815958), ('postag[:2]:Fc', 1.815958), ('word[-2:]:,', 1.815958), ('postag:Fc', 1.815958), ('postag[:2]:RG', 1.5741019999999999), ('postag:RG', 1.5741019999999999), ('postag:Fp', 1.3483909999999999), ('word.lower():.', 1.3483909999999999)], neg=[('word.istitle()', -5.9882960000000001), ('word.isupper()', -3.0104899999999999), ('postag:NP', -1.965192), ('postag[:2]:NP', -1.965192)], pos_remaining=15266, neg_remaining=4016), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-ORG', feature_weights=FeatureWeights(pos=[('word.isupper()', 3.3183609999999999), ('word.lower():efe', 2.3100559999999999), ('word[-3:]:EFE', 1.3843099999999999), ('word.istitle()', 1.321483), ('word[-2:]:FE', 1.2305189999999999), ('word.lower():gobierno', 1.2032099999999999), ('word[-3:]:rno', 0.86924100000000004), ('-1:word.lower():al', 0.71728099999999995), ('-1:word.lower():el', 0.712086), ('-1:word.lower():del', 0.68311200000000005), ('word[-2:]:PP', 0.66978899999999997), ('word[-3:]:PP', 0.66206500000000001), ('word.lower():pp', 0.66206500000000001)], neg=[('postag[:2]:SP', -1.4595560000000001), ('postag:SP', -1.4595560000000001), ('-1:word.isupper()', -0.90553600000000001), ('bias', -0.87653000000000003), ('-1:word.lower():en', -0.87110600000000005), ('-1:word.lower():de', -0.78678199999999998), ('-1:postag[:2]:VM', -0.71795100000000001)], pos_remaining=3809, neg_remaining=764), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-PER', feature_weights=FeatureWeights(pos=[('word.istitle()', 1.8260430000000001), ('postag:NP', 0.844804), ('postag[:2]:NP', 0.844804), ('+1:postag:VMI', 0.70768699999999995), ('-1:postag:VMI', 0.64877099999999999), ('-1:word.lower():a', 0.59973600000000005)], neg=[('postag:DA', -1.108144), ('postag[:2]:DA', -1.108144), ('-1:word.lower():del', -0.95387200000000005), ('-1:postag:DA', -0.90264500000000003), ('-1:postag[:2]:DA', -0.90264500000000003), ('-1:word.lower():la', -0.83193899999999998), ('word[-2:]:os', -0.71711499999999995), ('-1:word.istitle()

With `all_possible_transitions=True` CRF learned large negative weights for impossible transitions like O -> I-ORG.

It is also possible to format the result as text (could be useful in console):

In [10]:
print(format_as_text(expl))

Explained as: CRF

Transition features:
          B-LOC       O    B-ORG    B-PER    I-PER    B-MISC    I-ORG    I-LOC    I-MISC
------  -------  ------  -------  -------  -------  --------  -------  -------  --------
B-LOC    -0.133  -0.019   -0.753   -0.178   -1.331    -0.450   -1.462    3.305    -1.193
O         1.503   2.573    2.066    1.792   -5.007     1.333   -6.430   -5.120    -5.563
B-ORG    -0.187   0.225   -0.993   -0.226   -1.901    -0.585    4.358   -1.342    -1.954
B-PER    -0.824  -0.575   -0.933   -0.855    4.060    -0.556   -1.699   -1.104    -1.503
I-PER    -0.377  -0.879   -0.833   -0.592    3.053    -0.461   -1.235   -0.835    -0.985
B-MISC   -0.396  -0.560   -0.422   -0.351   -0.992    -0.372   -1.143   -0.759     4.372
I-ORG    -1.741  -0.310   -1.247   -0.270   -1.567    -0.723    4.737   -0.935    -1.481
I-LOC    -0.678  -0.198   -0.623   -0.486   -0.678    -0.385   -0.842    2.789    -0.593
I-MISC   -1.134  -0.511   -0.912   -0.414   -1.152    -0.455   -1.378 